# SENet

In [ ]:
import pickle
import torch


def load_state_dict(model, fname):
    """

    Arguments:
        model: model
        fname: file name of parameters converted from a Caffe model, assuming the file format is Pickle.
    """
    with open(fname, 'rb') as f:
        weights = pickle.load(f, encoding='latin1')

    own_state = model.state_dict()
    for name, param in weights.items():
        if name in own_state:
            try:
                own_state[name].copy_(torch.from_numpy(param))
            except Exception:
                raise RuntimeError('While copying the parameter named {}, whose dimensions in the model are {} and whose '\
                                   'dimensions in the checkpoint are {}.'.format(name, own_state[name].size(), param.size()))
        else:
            raise KeyError('unexpected key "{}" in state_dict'.format(name))

# Testing NN2 (SENet)

In [ ]:
from SENet import SENet
from utils import get_labels
model = SENet.senet50(num_classes=8631, include_top=True)

load_state_dict(model,'senet50_scratch_weight.pkl')
model.eval()

LABELS = get_labels()

In [ ]:
import os
import tensorflow as tf
import re
from torchvision import transforms
from utils import load_image_NN2

dataset_dir =  "test_set_cropped/"

correct_predictions = 0
total_images = 0

for filename in os.listdir(dataset_dir):
    if filename.endswith(".jpg") or filename.endswith(".jpeg"):
        person_path = os.path.join(dataset_dir, filename)
        img_test = load_image_NN2(person_path)
        with torch.no_grad():
            preds = model(img_test)
        
        pred_id = torch.argmax(preds, dim=1).item()

        pred_label = LABELS[pred_id]

        print("Immagine:", filename)
        print("Predizione del modello:", pred_label)
        correct_label = re.sub(r'_\d+_face_0\.jpg$', '', filename)
        print("Etichetta corretta:", correct_label)


        if correct_label in pred_label:
                correct_predictions += 1
                print("OK")
        '''
        else:
             with open('predizioni_errate.txt', 'a') as file:
              file.write(filename)
              file.write('\n')
        '''
        total_images += 1
        print("-------------------------->")


if total_images == 0:
    print("Nessuna immagine trovata nel dataset.")
else:
    test_accuracy = correct_predictions / total_images
    print('\n')
    print('\n')
    print("Test Accuracy:", test_accuracy)
    print("Numero di predizioni corrette:", correct_predictions)
    print("Numero di immagini processate:", total_images)

# Single immage test

In [ ]:
import torch
from PIL import Image
import numpy as np
import requests
import os
import torchvision.transforms as transforms




# Carica l'immagine
img = Image.open("test_set_cropped/Antonio_Cassano_10_face_0.jpg")

# Ridimensiona l'immagine a 224x224
img = img.resize((224, 224))

img = add_padding(img, (224,224))

# Converti l'immagine in un array NumPy
img = np.array(img)

# Mean BGR values
mean_bgr = [91.4953, 103.8827, 131.0912]

# Applica le trasformazioni


img_tensor = transform(img, mean_bgr)
img_tensor = img_tensor.unsqueeze(0)  # Aggiungi una dimensione batch

# Imposta il modello in modalità di valutazione
model.eval()

# Esegui l'inferenza
with torch.no_grad():
    preds = model(img_tensor)

pred_id = torch.argmax(preds, dim=1).item()

pred_label = LABELS[pred_id]

print('Predicted: ', pred_label)
